In [1]:
import skrf as rf
import numpy as npy
from skrf.constants import *

import skrf as rf
import numpy as npy
from skrf.constants import *

frequency = rf.Frequency(start=2000, stop=4000, npoints=1, unit='MHz')
ntwkAttn1_4000mK = rf.components.Attenuator(.05, frequency=frequency, T0 = 4)
ntwkAttn2_1000mK = rf.components.Attenuator(3, frequency=frequency, T0 = 1)
ntwkAttn_700mK = rf.components.Attenuator(10, frequency=frequency, T0 = .7)
ntwkAttn_400mk = rf.components.Attenuator(3, frequency=frequency, T0=.4)
ntwkAttn_100mk = rf.components.Attenuator(3, frequency=frequency, T0=.1)
ntwkCirc1 = rf.components.Circulator(0.5, frequency = frequency, T0 = .1)
ntwkCirc2 = rf.components.Circulator(0.5, frequency = frequency, T0 = .1)

ntwkC = rf.cascade_2port(ntwkAttn1_4000mK, ntwkAttn2_1000mK)

sys = rf.MultiNetworkSystem()

# add all the networks to the system
sys.add(ntwkAttn1_4000mK, r'attn_4K')
sys.add(ntwkAttn2_1000mK, r'attn_1K')
sys.add(ntwkAttn_700mK, r'attn_0p7K')
sys.add(ntwkAttn_400mk, r'attn_0p4K')
sys.add(ntwkAttn_100mk, r'attn_0p1K')
sys.add(ntwkCirc1, r'inj_circ')
sys.add(ntwkCirc1, r'jpa_circ')

# make all the connections between the elements
sys.connect(r'attn_4K', 2, r'attn_1K', 1)
sys.connect(r'attn_1K', 2, r'attn_0p7K', 1)
sys.connect(r'attn_0p7K', 2, r'attn_0p4K', 1)
sys.connect(r'attn_0p4K', 2, r'attn_0p1K', 1)
sys.connect(r'attn_0p1K', 2, r'inj_circ', 1)
sys.connect(r'inj_circ', 3, r'jpa_circ', 1)


# set the external port numbers, for this example the reduced network is a two-port network. The following is used to identify port 1 and port 2 of that resulting network.
sys.external_port(r'attn_4K', 1, external_port_number=1)
sys.external_port(r'inj_circ', 2, external_port_number=2)
sys.external_port(r'jpa_circ', 2, external_port_number=3)
sys.external_port(r'jpa_circ', 3, external_port_number=3)

# the verify function make sure that all the ports have been asigned and that everything is consistant for the reduction algorithm
(en, e) = sys.verify()
print(e)
for error in e:
    print(error)
print("Total number of Errors: " + str(en))

# calculate and return the resulting network
ntwk = sys.reduce()
print(ntwk)
print(20*npy.log10(npy.abs(ntwk.s)))
print(ntwk.cs)

{1: 'External ports must have correct assignments, e.g. 1, 2, 3, ... '}
1
Total number of Errors: 1
4-Port Network: '',  2000.0-2000.0 MHz, 1 pts, z0=[50.+0.j 50.+0.j 50.+0.j 50.+0.j]
[[[  -inf   -inf   -inf -20.05]
  [-19.55   -inf   -inf   -inf]
  [  -inf  -1.     -inf   -inf]
  [  -inf   -inf  -0.5    -inf]]]
[[[5.96049611e-24+0.j 0.00000000e+00+0.j 0.00000000e+00+0.j
   0.00000000e+00+0.j]
  [0.00000000e+00+0.j 2.08490012e-24+0.j 0.00000000e+00+0.j
   0.00000000e+00+0.j]
  [0.00000000e+00+0.j 0.00000000e+00+0.j 1.52717065e-25+0.j
   0.00000000e+00+0.j]
  [0.00000000e+00+0.j 0.00000000e+00+0.j 0.00000000e+00+0.j
   8.07492345e-26+0.j]]]
